# ROK_OCR

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import cv2

from driveHelper import GHelper

## 1. 이미지 -> BINARY_INV로

In [ ]:
def preprocess_pngs(data_path, save_path, verbose=False):
    png_list = glob.glob(data_path + '*.png')
    thr = 200
    try:
        os.mkdir(save_path)
    except FileExistsError:
        pass
    for file_path in png_list:
        img = cv2.imread(file_path,0)
        img = img[230:850,335:1400]
        ret,thresh2 = cv2.threshold(img,thr,255,cv2.THRESH_BINARY_INV)
        png_name = file_path.split('\\')[-1]
        save_name = save_path + png_name[:-4] + "_BINARY_INV.png"
        if verbose:
            print("FROM : {}".format(file_path))
            print("TO : {}".format(save_name))
        cv2.imwrite(save_name, thresh2)

In [ ]:
data_path = './upload_samples/'
save_path = './upload_samples/BIN_INV/'
preprocess_pngs(data_path, save_path, True)

## 2. 구글 드라이브를 통한 OCR

In [ ]:
gh = GHelper()

### 2.1 upload

In [ ]:
folder_id = '1LREb_uoCXV6ELURQW25fbfzEP01puwHt'
file_names = glob.glob(save_path + '*.png')
file_ids = []

In [ ]:
upload_len = len(file_names)
for i, file_name in enumerate(file_names):
    fn = os.path.basename(file_name)
    fp = os.path.dirname(file_name) + '/'
    print("Start Uploading ({}/{}) : {}".format(i+1, upload_len, fp+fn))
    file_id = gh.upload(fn, fp, folder_id=folder_id, as_docs=True, verbose=True)
    file_ids.append(file_id)

### 2.1 download

In [ ]:
save_path= './download_sample/'

In [ ]:
download_len = len(file_ids)
for i, file_id in enumerate(file_ids):
    print("Start Downloading ({}/{}) : {}".format(i+1, download_len, file_id))
    gh.download(file_id, save_path, verbose=True)

# text 후처리

In [ ]:
txt_path = './download_sample/'
txt_names = glob.glob(txt_path + '*.txt')

In [ ]:
ocr_results = []
for txt_name in txt_names:
    with open(txt_name, 'r', encoding='utf-8-sig') as f:
        a = f.readlines()
        f.close()
        a = list(map(str.strip, a))
        for i in reversed(range(len(a))):
            a[i] = ''.join(a[i].split(','))
            a[i] = ''.join(a[i].split('.'))
            if a[i].count('_') > 5:
                a.pop(i)
        ocr_results.append(a)

In [ ]:
ocr_results

# row sepearte 하고 OCR 해볼까?
# 시간 두배로 하고 col seperate도 진행

In [ ]:
image_name = './upload_samples/BIN_INV/Screenshot_2020-05-11-19-02-04_BINARY_INV.png'
img = cv2.imread(image_name,0)

height, width = img.shape

for x in range(1,6):
    fidx = int((x-1) * height/6)
    eid = int(x * height/6)
    plt.imshow(img[:,:])
    plt.show()
    line_idx = int(x * height/6)
    img[line_idx] = [1]*width

In [ ]:
for x in range(6):
    fidx = int(x * height/6)
    eidx = int((x+1) * height/6)
    plt.figure(figsize=(16,16))
    plt.imshow(img[fidx:eidx, :], 'gray')
    plt.show()

In [ ]:
img_name = img[:,:width//2]
img_power = img[:,width//2:]

plt.figure(figsize=(16,16))
plt.imshow(img_name, 'gray')
plt.show()

plt.figure(figsize=(16,16))
plt.imshow(img_power, 'gray')
plt.show()

In [ ]:
def seperate_pngs(data_path, save_path, verbose=False):
    png_list = glob.glob(data_path + '*.png')
    try:
        os.mkdir(save_path)
    except FileExistsError:
        pass
    for file_path in png_list:
        img = cv2.imread(file_path,0)
        height, width = img.shape
        
        # height sep
        for x in range(1,6):
            line_idx = int(x * height/6)
            img[line_idx] = [1]*width
            
        # width sep
        img_name = img[:,:width//2]
        img_power = img[:,width//2:]
        
        png_name = file_path.split('\\')[-1]
        save_name_name = save_path + png_name[:-4] + "_name.png"
        save_name_power= save_path + png_name[:-4] + "_power.png"
        if verbose:
            print("FROM : {}".format(file_path))
            print("TO : {}".format(save_name_name))
            print("TO : {}".format(save_name_power))
        cv2.imwrite(save_name_name, img_name)
        cv2.imwrite(save_name_power, img_power)

In [ ]:
data_path2 = './upload_samples/BIN_INV/'
save_path2 = './upload_samples/BIN_INV/sep/'
seperate_pngs(data_path2, save_path2, True)

### 2.1 upload

In [2]:
gh = GHelper()

In [ ]:
folder_id = '1QTMBb2yDIonB7YKhXLjQrTlUV5f1G8Ph'
file_names = glob.glob(save_path2 + '*.png')
file_ids = []

In [ ]:
upload_len = len(file_names)
for i, file_name in enumerate(file_names):
    fn = os.path.basename(file_name)
    fp = os.path.dirname(file_name) + '/'
    print("Start Uploading ({}/{}) : {}".format(i+1, upload_len, fp+fn))
    file_id = gh.upload(fn, fp, folder_id=folder_id, as_docs=True, verbose=True)
    file_ids.append(file_id)

### 2.1 download

In [ ]:
save_path= './download_sample/sep/'

In [ ]:
download_len = len(file_ids)
for i, file_id in enumerate(file_ids):
    print("Start Downloading ({}/{}) : {}".format(i+1, download_len, file_id))
    gh.download(file_id, save_path, verbose=True)

# text 후처리

In [ ]:
txt_path = './download_sample/sep/'
txt_names = glob.glob(txt_path + '*.txt')

In [ ]:
ocr_results2 = []
for txt_name in txt_names:
    with open(txt_name, 'r', encoding='utf-8-sig') as f:
        a = f.readlines()
        f.close()
        a = list(map(str.strip, a))
        for i in reversed(range(len(a))):
            a[i] = ''.join(a[i].split(','))
            a[i] = ''.join(a[i].split('.'))
            if a[i].count('_') > 5:
                a.pop(i)
        ocr_results2.append(a)

In [ ]:
txt_names

In [ ]:
ocr_results2

In [ ]:
for i, elem in enumerate(ocr_results2):
    if len(elem) != 7:
        print(i, len(elem), elem)

# 정확도 떨어져서 아예 한줄씩

In [ ]:
def seperate1x1(data_path, save_path, verbose=False):
    png_list = glob.glob(data_path + '*.png')
    try:
        os.mkdir(save_path)
    except FileExistsError:
        pass
    id_num = 0
    for file_path in png_list:
        img = cv2.imread(file_path, 0)
        height, width = img.shape

        png_name = file_path.split('\\')[-1]
        save_name = save_path + png_name[:-4]

        # height sep
        for x in range(6):
            fidx = int(x * height/6)
            eidx = int((x+1) * height/6)
            plt.imshow(img[fidx:eidx, :], 'gray')

            if verbose:
                print("FROM : {}".format(file_path))
                print("TO : {}".format(save_name + "_{}.png".format(id_num)))
            cv2.imwrite(save_name + "_{}.png".format(id_num), img[fidx:eidx, :])
            id_num += 1

In [ ]:
data_path3 = './upload_samples/BIN_INV/'
save_path3 = './upload_samples/BIN_INV/onexone/'
seperate1x1(data_path3, save_path3, True)

In [3]:
data_path3 = './upload_samples/BIN_INV/'
save_path3 = './upload_samples/BIN_INV/onexone/'

### 2.1 upload

folder_id invoces/rok_cor : '1LREb_uoCXV6ELURQW25fbfzEP01puwHt'

In [4]:
gh = GHelper()

In [5]:
folder_id = '1LREb_uoCXV6ELURQW25fbfzEP01puwHt'
file_names = glob.glob(save_path3 + '*.png')
file_ids = []

In [6]:
upload_len = len(file_names)
for i, file_name in enumerate(file_names):
    fn = os.path.basename(file_name)
    fp = os.path.dirname(file_name) + '/'
    print("Start Uploading ({}/{}) : {}".format(i+1, upload_len, fp+fn))
    file_id = gh.upload(fn, fp, folder_id=folder_id, as_docs=True, verbose=True)
    file_ids.append(file_id)

Start Uploading (1/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-01-45_BINARY_INV_0.png
File ID: 1djMdjfvyj_nwSHAtMl6gQJob8fvAmPi9wuNl8Vvpnjw
Start Uploading (2/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-01-45_BINARY_INV_1.png
File ID: 1LD4tF4Ag7aVaGQpwvwYQQdw1thxHXwYVHueEAkrINrA
Start Uploading (3/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-01-45_BINARY_INV_2.png
File ID: 1PwD66PLKdiosmqdf8mckA7W4lCK27Y68S_Xb51ONUrw
Start Uploading (4/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-01-45_BINARY_INV_3.png
File ID: 1EEgMrhU2_pFrU5ZE67NtXsbWNO3qMCvLWjf4ceCdL8g
Start Uploading (5/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-01-45_BINARY_INV_4.png
File ID: 1azRWKZUM2HsODyJHB7XfMnRSP1VJLGwvnXrXhfFYEmg
Start Uploading (6/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-01-45_BINARY_INV_5.png
File ID: 1j7IcZTh-qBQYjPn3jDndo8TxpzQLsDNqn0_s3qkBZu4
Start Uploading (7/138) : ./upload

File ID: 1JyojoMRaO_IbbjKB-SGPc9_Q7H14fKbE8hgJxP36MLM
Start Uploading (52/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-02-26_BINARY_INV_51.png
File ID: 1WxmOUTB3M_JMnfFxC_8vmAIBW9mFHRvAgnWQO4ubqBk
Start Uploading (53/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-02-26_BINARY_INV_52.png
File ID: 1p9O1VsjPxKxeAYGkmcL6cwfqP2s-piHu0JmlbYHPzgg
Start Uploading (54/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-02-26_BINARY_INV_53.png
File ID: 1FSsgfYUfhOR5uM9Mr8q9sJGRvxqA6eX72F-oQikY8pg
Start Uploading (55/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-02-30_BINARY_INV_54.png
File ID: 1ms-rOHuocT-VN9HiiqYRqdiNk6MLtx2wQpiNkub_Uto
Start Uploading (56/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-02-30_BINARY_INV_55.png
File ID: 1A04sLJyVXF9FGWDeAVaB23AjavJHk_cRuEHJhc4DcBM
Start Uploading (57/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-02-30_BINARY_INV_56.png
File ID: 13T_AsyA0e-lr

File ID: 1-WkurDel7JOmNhMixruU_zSCY6ck2153A8Lptm8D5ek
Start Uploading (103/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-03-04_BINARY_INV_102.png
File ID: 1JXlfUROy-ftU5XPcB-805qQLcxRyctlf-Iebri_My7k
Start Uploading (104/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-03-04_BINARY_INV_103.png
File ID: 1UGCdCQAIzfxEquiDEoh85xeZZFC3SNSS2GNSmlwW3vE
Start Uploading (105/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-03-04_BINARY_INV_104.png
File ID: 1xoqA8uv5p-Zeopfr-_hVMuVHbHAohynL0_mbXu0UpEQ
Start Uploading (106/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-03-04_BINARY_INV_105.png
File ID: 1Tm7le2rGH7-xvOFrePdopCEe7fo3I8TliNg0r_Tefz0
Start Uploading (107/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-03-04_BINARY_INV_106.png
File ID: 1aa1_W91NcAL9h5Q3nok3dXI3HTaD0pAwaJ6jcr2iXzQ
Start Uploading (108/138) : ./upload_samples/BIN_INV/onexone/Screenshot_2020-05-11-19-03-04_BINARY_INV_107.png
File ID: 1

### 2.1 download

In [7]:
save_path= './download_sample/onexone/'

In [8]:
download_len = len(file_ids)
for i, file_id in enumerate(file_ids):
    print("Start Downloading ({}/{}) : {}".format(i+1, download_len, file_id))
    gh.download(file_id, save_path, verbose=True)

Start Downloading (1/138) : 1djMdjfvyj_nwSHAtMl6gQJob8fvAmPi9wuNl8Vvpnjw
Screenshot_2020-05-11-19-01-45_BINARY_INV_0.txt Download 100%.
Start Downloading (2/138) : 1LD4tF4Ag7aVaGQpwvwYQQdw1thxHXwYVHueEAkrINrA
Screenshot_2020-05-11-19-01-45_BINARY_INV_1.txt Download 100%.
Start Downloading (3/138) : 1PwD66PLKdiosmqdf8mckA7W4lCK27Y68S_Xb51ONUrw
Screenshot_2020-05-11-19-01-45_BINARY_INV_2.txt Download 100%.
Start Downloading (4/138) : 1EEgMrhU2_pFrU5ZE67NtXsbWNO3qMCvLWjf4ceCdL8g
Screenshot_2020-05-11-19-01-45_BINARY_INV_3.txt Download 100%.
Start Downloading (5/138) : 1azRWKZUM2HsODyJHB7XfMnRSP1VJLGwvnXrXhfFYEmg
Screenshot_2020-05-11-19-01-45_BINARY_INV_4.txt Download 100%.
Start Downloading (6/138) : 1j7IcZTh-qBQYjPn3jDndo8TxpzQLsDNqn0_s3qkBZu4
Screenshot_2020-05-11-19-01-45_BINARY_INV_5.txt Download 100%.
Start Downloading (7/138) : 1BPZhK4BB5Wik2GMYJidrNt_h2kggWlaUpbz4WyHFmF0
Screenshot_2020-05-11-19-02-00_BINARY_INV_10.txt Download 100%.
Start Downloading (8/138) : 1AwcZk3KRTZcHAOdYoU

Screenshot_2020-05-11-19-02-30_BINARY_INV_59.txt Download 100%.
Start Downloading (61/138) : 11zuXLYbw3PukBpV4pEpnDSIgH8KS33lPSdxfpzTo0ss
Screenshot_2020-05-11-19-02-38_BINARY_INV_60.txt Download 100%.
Start Downloading (62/138) : 1j4BsNf75SjlDizrBN5qTwJJzJBKOx2yyhLmCa_3Gbs4
Screenshot_2020-05-11-19-02-38_BINARY_INV_61.txt Download 100%.
Start Downloading (63/138) : 1-Hcm4RS89_3LcTjX4Y-T1HEZCHTAIWutA6AGiYz9W6A
Screenshot_2020-05-11-19-02-38_BINARY_INV_62.txt Download 100%.
Start Downloading (64/138) : 1xBhAEVMrWywFwqev2phmuyIfNf-2QCU4bb3m4pe-OW0
Screenshot_2020-05-11-19-02-38_BINARY_INV_63.txt Download 100%.
Start Downloading (65/138) : 18mQWzv-zEVRPSNnfbaBAearuyUhsYkVyl_mC4x5Nvs8
Screenshot_2020-05-11-19-02-38_BINARY_INV_64.txt Download 100%.
Start Downloading (66/138) : 1RixuUKfv2iiaQW_QYKgd9KCNcPyLSGJjisGbRFcXffs
Screenshot_2020-05-11-19-02-38_BINARY_INV_65.txt Download 100%.
Start Downloading (67/138) : 1I-XpYYTnkenkVAWQWJRpGorfpkfPV-fZa3TobUWuNQQ
Screenshot_2020-05-11-19-02-42_BIN

Screenshot_2020-05-11-19-03-11_BINARY_INV_119.txt Download 100%.
Start Downloading (121/138) : 1Rv--CyD9hP6OvOJBR-2K8sud0rGqTkyGudLNCngBopM
Screenshot_2020-05-11-19-03-16_BINARY_INV_120.txt Download 100%.
Start Downloading (122/138) : 1kGEGFShbbwd92WQ_ha3F0qiIijt0Q5Go2IZ_WqU1MWI
Screenshot_2020-05-11-19-03-16_BINARY_INV_121.txt Download 100%.
Start Downloading (123/138) : 111UDCdGI8nvWmzmbNazkpqSah9SeZZCjWDbiG9-pUkA
Screenshot_2020-05-11-19-03-16_BINARY_INV_122.txt Download 100%.
Start Downloading (124/138) : 18Yu0NuZd4IrNkbZhV2iaV5AknsYtC7VDeOaKNLOjLWI
Screenshot_2020-05-11-19-03-16_BINARY_INV_123.txt Download 100%.
Start Downloading (125/138) : 1L-EsGQGvIYXhiWWk6huj5Lf3WeeXnubNBbzATf7kxqk
Screenshot_2020-05-11-19-03-16_BINARY_INV_124.txt Download 100%.
Start Downloading (126/138) : 1i8_7wHha4Wpc3eZHfhTC_e1di7k5kfyKxoRE8DthX0c
Screenshot_2020-05-11-19-03-16_BINARY_INV_125.txt Download 100%.
Start Downloading (127/138) : 1_iIKUh_f-Gvsw7-C9DANngK9vHlxqu4aRARSbAuvySQ
Screenshot_2020-05-1

# text 후처리

In [16]:
os.path.basename(txt_names[0]).split('.')[0].split('_')[-1]

'0'

In [9]:
txt_path = './download_sample/onexone/'
txt_names = glob.glob(txt_path + '*.txt')

In [25]:
ocr_results3 = []
for txt_name in txt_names:
    file_num = os.path.basename(txt_name).split('.')[0].split('_')[-1]
    with open(txt_name, 'r', encoding='utf-8-sig') as f:
        a = f.readlines()
        f.close()
        a = list(map(str.strip, a))
        for i in reversed(range(len(a))):
            a[i] = ''.join(a[i].split(','))
            a[i] = ''.join(a[i].split('.'))
            if a[i].count('_') > 5:
                a.pop(i)
        if not a[0]:
            a.pop(0)
        a.append(file_num)
        ocr_results3.append(a)

In [26]:
ocr_results3

[['폭풍검 맹주', '92766209', '0'],
 ['외돌', '64868740', '1'],
 ['Big Sish', '62022496', '2'],
 ['三國 알콜홀릭 장비', '57306656', '3'],
 ['폭풍 돌팸 황충', '54904940', '4'],
 ['방범은순찰', '54080839', '5'],
 ['유감토깽이', '44283716', '10'],
 ['불', '곰', '43482552', '-', '11'],
 ['행성파괴범', '50473307', '6'],
 ['北 안감독', '48509544', '7'],
 ['6072리', '47257891', '8'],
 ['폭풍JOKER 성인', '45310635', '9'],
 ['수정', '43332529', '12'],
 ['유로', '42464132', '13'],
 ['포유', '40871073', '14'],
 ['40248685', '15'],
 ['꽃', '0009', '40025455', '16'],
 ['찬사 대사', '39847285', '17'],
 ['북덕북덕', '39518411', '18'],
 ['北 서태웅', '38886046', '19'],
 ['혈마', '38096739', '20'],
 ['꼬따발', '37858456', '명장', '21'],
 ['폭풍을지 고문', '37381148', '22'],
 ['폭풍 캠핑', '37166287', '23'],
 ['폭풍 청산', '36809482', '24'],
 ['아버', '36443701', '25'],
 ['36210964', '26'],
 ['액트', '36038023', '27'],
 ['미산M', '36033619', '28'],
 ['약먹', '35703165', '29'],
 ['조시안서', '35630079', '30'],
 ['fireworkcorki', '35364942', '31'],
 ['우강다', '35267342', '32'],
 ['기범아마', '35199968', '33']

In [27]:
for i, elem in enumerate(ocr_results3):
    if len(elem) != 3:
        print(i, len(elem), elem)

7 5 ['불', '곰', '43482552', '-', '11']
15 2 ['40248685', '15']
16 4 ['꽃', '0009', '40025455', '16']
21 4 ['꼬따발', '37858456', '명장', '21']
26 2 ['36210964', '26']
39 2 ['33774111', '39']
59 2 ['30332361', '59']
89 4 ['26160236', 'Dragonyy', '26460236', '89']
